In [ ]:
!pip install langchain langchain-openai langchain-pinecone langchain-community

In [2]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
os.environ['OPENAI_EMBEDDING_MODEL'] = userdata.get('OPENAI_EMBEDDING_MODEL')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
import requests
from pprint import pprint

end_point = 'https://apis.data.go.kr/1471000/HtfsInfoService03'
encoding = 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA%3D%3D'
decoding = 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA=='

url = 'http://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01?'

# 3) 요청 변수(params) 세팅
params = {
    "pageNo":    "1",          # 페이지 번호
    "numOfRows": "10",         # 한 페이지 결과 수
    "ServiceKey": 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA==', # Swagger에 표시된 정확한 이름(ServiceKey)
    "type":      "json",       # 응답 포맷(xml/json) – default: xml
    # (필요시 API별 추가 파라미터를 여기에 더합니다)
}

# 4) 실제 요청 보내기
response = requests.get(url, params=params, timeout=10)
response.raise_for_status()

# 5) 결과 확인
pprint(response.json())

{'body': {'items': [{'item': {'BASE_STANDARD': '1. 성상 : 고유의 향미가 있고 이미·이취가 없는 '
                                               '노랑 하양색의 입자성이 있는 분말\n'
                                               '2. 프로바이오틱스 수(표시량 '
                                               '300,000,000,000(3,000억) CFU/g) '
                                               ': 표시량 이상\n'
                                               '3. 대장균군 : 음성\n'
                                               '4. 납 : 1.0 mg/kg 이하\n'
                                               '5. 카드뮴 : 0.3 mg/kg 이하',
                              'DISTB_PD': '제조일로부터 24개월까지',
                              'ENTRPS': '일동바이오사이언스(주)',
                              'INTAKE_HINT1': '1. 질환이 있거나 의약품 복용 시 전문가와 '
                                              '상담하십시오.\n'
                                              '2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 '
                                              '수 있습니다.\n'
                                              '3. 어

In [ ]:
# 객체 생성
api_json = response.json()

## JSON → 문서(Document)

In [ ]:
from langchain_core.documents import Document

def json_to_documents(api_json: dict) -> list[Document]:
    documents = []

    for entry in api_json['body']['items']:
        data = entry['item']

        # page_content 구성
        text = f"""
제품명: {data.get('PRDUCT', '').strip()}
제조사: {data.get('ENTRPS', '').strip()}
기능성: {data.get('MAIN_FNCTN', '').strip()}
섭취 시 주의사항: {data.get('INTAKE_HINT1', '').strip()}
보관조건: {data.get('PRSRV_PD', '').strip()}
유통기한: {data.get('DISTB_PD', '').strip()}
"""

        # metadata 구성
        metadata = {
            "STTEMNT_NO": data.get("STTEMNT_NO"),
            "제조사": data.get("ENTRPS"),
            "기준규격": data.get("BASE_STANDARD", "").strip()
        }

        # Document 생성
        documents.append(Document(page_content=text, metadata=metadata))

    return documents

In [ ]:
documents = json_to_documents(api_json)

print(documents[0].page_content)
print(documents[0].metadata)


제품명: 11종 혼합유산균
제조사: 일동바이오사이언스(주)
기능성: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음
섭취 시 주의사항: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.
2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.
3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오.
4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
보관조건: 냉장조건
유통기한: 제조일로부터 24개월까지

{'STTEMNT_NO': '20140017002183', '제조사': '일동바이오사이언스(주)', '기준규격': '1. 성상 : 고유의 향미가 있고 이미·이취가 없는 노랑 하양색의 입자성이 있는 분말\n2. 프로바이오틱스 수(표시량 300,000,000,000(3,000억) CFU/g) : 표시량 이상\n3. 대장균군 : 음성\n4. 납 : 1.0 mg/kg 이하\n5. 카드뮴 : 0.3 mg/kg 이하'}


##  HuggingFace 임베딩 모델 로드

In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings
# import torch

# embedding = HuggingFaceEmbeddings(
#     model_name="jhgan/ko-sbert-sts",   # 한국어 RAG에 자주 쓰이는 Sentence-BERT 기반 모델
#     model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
# )

In [ ]:
# 문서로부터 벡터스토어 생성
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# 임베딩 모델
## 한방에 밀어넣으면 오류 날 수 있으므로 batch_size로 나누서 text로 고려해보기
embeddings = OpenAIEmbeddings(model=os.environ['OPENAI_EMBEDDING_MODEL'])

vector_store = PineconeVectorStore.from_documents(
    documents,
    embedding=embeddings,
    index_name='health-supplement-rag1'
)


## + Pinecone 벡터 DB에 업로드

In [ ]:
# 존재하는 인덱스에 접근/검색
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
vector_store = PineconeVectorStore(
    index=pc.Index('health-supplement-rag1'),
    embedding=embeddings
)

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3} # 유사한 문서 3개까지 검색
)
print(retriever.invoke('피로개선에 도움이 되는 영양제는?'))

[Document(id='4b772d8e-05bc-4c94-95e3-0cc70b835e5d', metadata={'STTEMNT_NO': '20040017038206', '기준규격': '① 성상 : 고유의 향미가 있고 이미ㆍ이취가 없는 흑갈색의 유동성 액체\n② 진세노사이드 Rg1, Rb1 및 Rg3의 합 : 표시량(23mg/g)이상\n③ 세균수(1 ㎖당) : 3000이하\n④ 대장균군 : 음성\n⑤ 납 : 1.0 mg/kg 이하\n⑥ 카드뮴 : 0.3 mg/kg 이하', '제조사': '케이지랩 주식회사'}, page_content='\n제품명: 고려홍삼농축액(23mg/g)\n제조사: 케이지랩 주식회사\n기능성: ①면역력 증진에 도움을 줄 수 있음 \n②피로개선에 도움을 줄 수 있음 \n③혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음 \n④기억력 개선에 도움을 줄 수 있음 \n⑤항산화에 도움을 줄 수 있음\n섭취 시 주의사항: (가)당뇨치료제 및 혈액항응고제 복용 시 섭취에 주의\n(나)알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음\n(다)이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것\n보관조건: 습기와 직사광선을 피하여 서늘한 곳에 마개를 막아 보관하십시오.\n유통기한: 제조일로부터 36개월까지\n'), Document(id='a64bc977-92d3-4c84-a553-2ed45a480bf6', metadata={'STTEMNT_NO': '2004001703094', '기준규격': '1. 성상 : 황갈색의 입자상 분말\r\n2. 진세노사이드 Rg1, Rb1 및 Rg3의 합 : 표시량(10.5mg/g) 이상\r\n3. 대장균군 : 음성', '제조사': '(주)그린바이오'}, page_content='\n제품명: Puffing red ginseng extract powder-J\n제조사: (주)그린바이오\n기능성: [홍삼(원료성)]①면역력 증진에 도움을 줄 수 있음②피로개선에 도움을 줄 수 있음③혈소판 응집 억제를 통한 혈액흐름에 도

## RAG 구성 (Retriever + LLM → 최종 답변 생성) 단계

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4.1-mini",  #
    temperature=0.3, # 0.3 답변이 신중하고 일관
    max_tokens=512
)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,           # Step 4에서 만든 것
    chain_type="stuff",            # "stuff"는 검색 결과를 LLM에 다 넘김
    return_source_documents=True   # 🔍 근거 문서 포함 응답
)

In [ ]:
query = "Ento Probiotics plus Blood sugar care에 대해 알려줘"

response = qa_chain(query)

print("🧠 GPT 응답:")
print(response['result'])
print("\n📎 참고 문서:")
for doc in response['source_documents']:
    print(doc.page_content)
    print("-" * 60)

🧠 GPT 응답:
Ento Probiotics plus Blood sugar care는 주식회사 아람에서 제조한 제품으로, 전량 수출용입니다. 이 제품의 주요 기능성은 다음과 같습니다:

- 유산균 증식 및 유해균 억제
- 배변활동 원활
- 장 건강에 도움을 줄 수 있음
- 식후 혈당상승 억제에 도움을 줄 수 있음

또한, 이 제품은 정상적인 면역기능과 정상적인 세포분열에 필요한 성분을 포함하고 있습니다.

섭취 시 주의사항은 다음과 같습니다:
- 질환이 있거나 의약품 복용 시 전문가와 상담할 것
- 알레르기 체질 등 개인에 따라 과민반응이 나타날 수 있음
- 어린이가 함부로 섭취하지 않도록 일일 섭취량 방법을 지도할 것
- 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것

보관은 직사광선 및 고온다습한 곳을 피하고 서늘한 곳에 하며, 어린이의 손에 닿지 않는 곳에 보관해야 합니다. 유통기한은 제조일부터 24개월입니다.

📎 참고 문서:

제품명: Ento Probiotics plus Blood sugar care (전량수출용)
제조사: 주식회사 아람
기능성: 유산균 증식 및 유해균 억제･배변활동 원활･장 건강에 도움을 줄 수 있음

① 식후 혈당상승 억제에 도움을 줄 수 있음

(1) 정상적인 면역기능에 필요
(2) 정상적인 세포분열에 필요
섭취 시 주의사항: (가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것
(나) 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있음
(다) 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도할 것
(라) 이상사례 발생 시 섭취를 중단하고 전문가와 상담할 것
보관조건: 직사광선 및 고온다습한 곳을 피하고 서늘한 곳에 보관한다. 어린이의 손에 닿지 않는 곳에 보관하십시오.
유통기한: 제조일부터 24개월

------------------------------------------------------------

제품명: KA-NE PLUS ZINC (수출용)
제조사: 에프엔바이오(주)
기능성: ①탄수화물

### (전처리) 표준화 된 키워드 제공

In [ ]:
# # 증상 → 기능성 키워드 사전
# symptom_to_function = {
#     "변비": "배변활동",
#     "설사": "장 건강",
#     "장 트러블": "장 건강",
#     "더부룩함": "장 건강",
#     "피곤함": "피로 개선",
#     "눈 피로": "눈 건강",
#     "시력 저하": "눈 건강",
#     "갱년기": "갱년기 여성 건강",
#     "혈액순환": "혈행 개선",
#     "콜레스테롤": "혈중 중성지질 개선",
#     "기억력": "기억력 개선",
#     "면역력": "면역력 증진",
#     "피부 트러블": "피부 건강",
#     "간": "간 건강",
# }

# # 매핑 함수
# def map_symptom_to_function(user_input: str) -> str:
#     for symptom, function in symptom_to_function.items():
#         if symptom in user_input:
#             return function
#     return user_input  # 기본 fallback: 그대로 반환

In [ ]:
# query = "장 트러블이 심한데 어떤 제품이 좋아?"

# mapped_query = map_symptom_to_function(query)
# print(f"🧾 정규화된 쿼리: {mapped_query}")

# # → 출력: "장 건강"

🧾 정규화된 쿼리: 장 건강


In [ ]:
# #  RAG 파이프라인 연결
# response = qa_chain.run(mapped_query)
# print(response)